In [1]:
#Imports
import sys
sys.path.append('../python/')
import NGC5533_functions as nf

import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt
import lmfit as lm
import dataPython as dp

In [2]:
data = dp.getXYdata_wXYerr('data/NGC5533-rot-data_fmt.txt')
r_dat = np.asarray(data['xx'])
v_dat = np.asarray(data['yy'])
v_err0 = np.asarray(data['ex'])
v_err1 = np.asarray(data['ey'])

In [3]:
def f(r,M,rc,rho00,c,d):
    return np.sqrt(nf.bh_v(r,M)**2 + nf.h_v(r,rc,rho00)**2 + c*nf.b_v(r,load=True)**2 + d*nf.d_v(r,load=True)**2)
#Next: Spline bulge, make that a function, give prefactor
# - call file, get vector
# - spline fit vector
# - calculate over spline
# - prefactor in sum
#Probably similar thing for disk
#Eventually fit for n?

#Cache/memoize
# - number of iterations -> number of function calls
# - 40 calls was reasonable

#Save functions/vectors at certain points rather than individual values
#Save/load both X and Y

In [4]:
#Setup
weighdata = 1/v_err1
l_mod = lm.Model(f)
params = l_mod.make_params(M=nf.Mbh_def,rc=nf.h_rc,rho00=nf.h_rho00,c=1,d=1)

#Do fit
l_fit = l_mod.fit(v_dat[3::],params,r=r_dat[3::],weights=weighdata)

#Define Stuff
l_dict = l_fit.best_values
l_M = l_dict['M']
l_rc = l_dict['rc']
l_rho00 = l_dict['rho00']
l_c = l_dict['c']
l_d = l_dict['d']

i = 0
l_curve = np.zeros(len(r_dat))
while i < len(r_dat):
    l_curve[i] = np.sqrt(nf.bh_v(r_dat,l_M)[i]**2 + nf.h_v(r_dat,l_rc,l_rho00)[i]**2 + l_c*nf.b_v(r_dat,load=True)[i]**2 + l_d*nf.d_v(r_dat,load=True)[i]**2)
    i += 1
    
d_curve = l_d*nf.d_v(r_dat,load=True)

i = 0
nodisk = np.zeros(len(r_dat))
while i < len(r_dat):
    nodisk[i] = np.sqrt(nf.bh_v(r_dat,l_M)[i]**2 + nf.h_v(r_dat,l_rc,l_rho00)[i]**2 + l_c*nf.b_v(r_dat,load=True)[i]**2)
    i += 1

No such group! Aborting.


UnboundLocalError: local variable 'dset' referenced before assignment

In [ ]:
fig = plt.figure(figsize=(9.0,8.0))

plt.errorbar(r_dat,v_dat,yerr=v_err1,fmt='bo',label='Data')
plt.plot(r_dat,l_curve,label='LmFit')
plt.plot(r_dat,d_curve,label='Disk Only')
plt.plot(r_dat,nodisk,label='No Disk')

plt.legend()
plt.show()

In [ ]:
l_fit

In [ ]:
nf.d_v(r_dat,save=True)

In [5]:
nf.savedata([0,0],[1,1],'test','0011')

OSError: Unable to create file (unable to open file: name = './Inputs.hdf5', errno = 17, error message = 'File exists', flags = 15, o_flags = c2)